# 🎯 Iteracja 4: Few-Shot Learning

**Czas:** ~45 minut  
**Poziom:** Średniozaawansowany

---

## Cel iteracji

Poprawić accuracy klasyfikacji przez **pokazanie modelowi przykładów** poprawnych klasyfikacji - i sprawdzić ile przykładów naprawdę potrzebujemy.

---

## Teoria: Zero/One/Few-Shot Learning

**Shot** = przykład w prompcie.

| Typ | Opis | Kiedy używać |
|-----|------|--------------|
| **Zero-shot** | Brak przykładów | Prosta klasyfikacja, gdy model dobrze zna dziedzinę |
| **One-shot** | 1 przykład | Gdy chcemy ustalić format odpowiedzi |
| **Few-shot** | 3-8 przykładów | Gdy model nie radzi sobie zero-shot, trudne kategorie |
| **Many-shot** | 10+ przykładów | Bardzo specjalistyczne zadania, unikalne kategorie |

### Jak dobierać przykłady?

Jakość przykładów jest ważniejsza niż ich liczba:

1. **Reprezentatywność** - pokryj różne typy recenzji (krótkie, długie, wieloznaczne)
2. **Trudne przypadki** - dodaj przykłady które model klasyfikuje źle (analiza błędów!)
3. **Różnorodność** - nie dawaj 5x tego samego wzorca
4. **Graniczne przypadki** - przykłady między kategoriami (edge cases)
5. **Kolejność ma znaczenie** - ostatnie przykłady mają większy wpływ (recency bias)

### Few-shot vs Fine-tuning

| | Few-shot | Fine-tuning |
|--|---------|-------------|
| Koszt | Darmowy | Drogi |
| Szybkość | Natychmiastowy | Godziny/dni treningu |
| Wymagane dane | 3-10 przykładów | Setki/tysiące |
| Jakość | Dobra | Lepsza przy dużych danych |
| Elastyczność | Łatwa zmiana | Trzeba trenować ponownie |

> **Wniosek:** Zacznij zawsze od few-shot. Few-tuning tylko jeśli few-shot nie wystarcza i masz dużo danych.

## ⚙️ Setup - środowisko

Ta komórka wykrywa czy jesteś w **Google Colab** czy lokalnie, a następnie:
1. Klonuje repozytorium z GitHub (tylko Colab)
2. Instaluje wymagane biblioteki
3. Konfiguruje ścieżki importów

In [ ]:
import sys
import subprocess
from pathlib import Path

# Wykryj środowisko
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

print(f"{'🟡 Google Colab' if IN_COLAB else '💻 Lokalne środowisko'}")

# ============================================================
# KONFIGURACJA REPO (zmień URL na właściwe przed warsztatami)
# ============================================================
REPO_URL = "https://github.com/JSerek/techland-genai-workshop.git"
REPO_DIR = Path("/content/szkolenie_techland") if IN_COLAB else Path(".").resolve().parent
# ============================================================

if IN_COLAB:
    if not REPO_DIR.exists():
        print(f"📥 Klonuję repo...")
        subprocess.run(["git", "clone", REPO_URL, str(REPO_DIR)], check=True)
        print("✅ Repo sklonowane")
    else:
        print("🔄 Aktualizuję repo (git pull)...")
        subprocess.run(["git", "-C", str(REPO_DIR), "pull"], check=True)

    # Dodaj repo do sys.path
    sys.path.insert(0, str(REPO_DIR))

    # Zainstaluj zależności
    print("📦 Instaluję biblioteki...")
    subprocess.run(
        [sys.executable, "-m", "pip", "install", "-r", str(REPO_DIR / "requirements.txt"), "-q"],
        check=True
    )
    print("✅ Biblioteki zainstalowane")
else:
    # Lokalnie: dodaj root projektu do sys.path
    sys.path.insert(0, str(REPO_DIR))
    print(f"📂 Ścieżka projektu: {REPO_DIR}")

print("\n✅ Setup zakończony")

In [ ]:
# ============================================================
# 🔑 Konfiguracja API
# W Google Colab: dodaj sekrety w menu po lewej (🔑 ikona)
#   Nazwy sekretów: VERTEX_AI_API_KEY, VERTEX_AI_BASE_URL, MODEL_NAME
#
# Lokalnie: ustaw zmienne środowiskowe lub wpisz wartości bezpośrednio
# ============================================================

if IN_COLAB:
    from google.colab import userdata
    API_KEY   = userdata.get("VERTEX_AI_API_KEY")
    BASE_URL  = userdata.get("VERTEX_AI_BASE_URL")
    MODEL_NAME = userdata.get("MODEL_NAME") or "google/gemini-2.5-flash-lite"
else:
    import os
    API_KEY   = os.environ.get("VERTEX_AI_API_KEY", "TODO: wklej API key")
    BASE_URL  = os.environ.get("VERTEX_AI_BASE_URL", "TODO: wklej endpoint URL")
    MODEL_NAME = os.environ.get("MODEL_NAME", "google/gemini-2.5-flash-lite")

# Walidacja
if not API_KEY or API_KEY == "TODO: wklej API key":
    print("❌ Brak API_KEY! Dodaj sekret 'VERTEX_AI_API_KEY' w Colab Secrets.")
elif not BASE_URL or BASE_URL == "TODO: wklej endpoint URL":
    print("❌ Brak BASE_URL! Dodaj sekret 'VERTEX_AI_BASE_URL' w Colab Secrets.")
else:
    print(f"✅ API skonfigurowane: {MODEL_NAME}")
    print(f"   Endpoint: {BASE_URL[:60]}...")

In [ ]:
# Konfiguracja klienta LLM - dane pobrane z Secrets w komórce powyżej
from src.utils.llm_client import create_client, LLMProvider

PROVIDER = LLMProvider.VERTEX_AI  # Zmień jeśli używasz innego providera

client = create_client(
    provider=PROVIDER,
    api_key=API_KEY,
    base_url=BASE_URL,
)
print(f"✅ Klient LLM gotowy: {PROVIDER.value} / {MODEL_NAME}")

In [ ]:
import json
import pandas as pd
from pathlib import Path
from pydantic import BaseModel, Field
from typing import Optional

from src.evaluation.evaluator import evaluate_trial, compare_trials, MatchStrategy


# Kategorie tematyczne
CATEGORIES = [
    "gameplay",       # Mechaniki rozgrywki, walka, parkour, sterowanie, balans
    "story",          # Fabuła, postacie, dialogi, scenariusz
    "bugs",           # Crashe, glitche, błędy, problemy techniczne
    "performance",    # FPS, lag, stuttering, optymalizacja ładowania
    "content",        # Ilość contentu, długość gry, powtarzalność
    "graphics",       # Grafika, wizualia, animacje
    "audio",          # Muzyka, dźwięki, voice acting
    "price",          # Cena, wartość za pieniądze, DLC
]
categories_str = ', '.join(f'"{{c}}"' for c in CATEGORIES)

print('✅ Biblioteki załadowane')
print(f'Kategorie ({len(CATEGORIES)}): {categories_str}')


In [ ]:
DATA_DIR = REPO_DIR / 'data'

# Wczytaj golden dataset
golden_path = DATA_DIR / 'evaluation' / 'golden_dataset.json'
with open(golden_path) as f:
    golden_data = json.load(f)

golden_texts  = [item['review_text'] for item in golden_data]
golden_labels = [item['labels']      for item in golden_data]

print(f'✅ Golden dataset: {len(golden_data)} recenzji')
print(f'   Kategorie: {sorted(set(l for item in golden_data for l in item["labels"]))}')

## 📋 Golden Dataset

Poniżej 100 recenzji z przypisanymi kategoriami — to nasz punkt odniesienia do ewaluacji.
Model będzie klasyfikował te same recenzje, a my sprawdzimy czy jego wyniki zgadzają się z etykietami.

| Kategoria | Opis |
|-----------|------|
| `gameplay` | Mechaniki rozgrywki, walka, parkour, sterowanie, balans |
| `story` | Fabuła, postacie, dialogi, scenariusz |
| `bugs` | Crashe, glitche, błędy, problemy techniczne |
| `performance` | FPS, lag, stuttering, optymalizacja |
| `content` | Ilość contentu, długość gry, powtarzalność |
| `graphics` | Grafika, wizualia, animacje |
| `audio` | Muzyka, dźwięki, voice acting |
| `price` | Cena, wartość za pieniądze, DLC |

In [ ]:
from IPython.display import display, HTML

def show_golden_dataset(data: list[dict], max_text: int = 120) -> None:
    """Wyświetla golden dataset jako sformatowaną tabelę HTML."""
    rows = []
    for item in data:
        text = item['review_text']
        if len(text) > max_text:
            text = text[:max_text] + '...'
        # Każda etykieta jako kolorowy badge
        COLORS = {
            'gameplay':    '#4C72B0', 'story':       '#DD8452',
            'bugs':        '#55A868', 'performance': '#C44E52',
            'content':     '#8172B2', 'graphics':    '#937860',
            'audio':       '#DA8BC3', 'price':       '#8C8C8C',
        }
        badges = ' '.join(
            f'<span style="background:{COLORS.get(l, "#888")};color:white;'
            f'padding:2px 8px;border-radius:10px;font-size:11px;">{l}</span>'
            for l in item['labels']
        )
        rows.append(f'<tr><td style="text-align:center;color:#888;">{item["id"]}</td>'
                    f'<td style="font-size:12px;">{text}</td>'
                    f'<td>{badges}</td></tr>')

    html = f'''
    <style>
        .golden-table {{ border-collapse: collapse; width: 100%; font-family: sans-serif; }}
        .golden-table th {{ background: #2d2d2d; color: white; padding: 8px 12px; text-align: left; }}
        .golden-table td {{ padding: 6px 12px; border-bottom: 1px solid #eee; vertical-align: top; }}
        .golden-table tr:hover td {{ background: #f9f9f9; }}
    </style>
    <table class="golden-table">
        <thead><tr>
            <th style="width:40px">#</th>
            <th>Recenzja</th>
            <th style="width:200px">Kategorie</th>
        </tr></thead>
        <tbody>{''.join(rows)}</tbody>
    </table>
    '''
    display(HTML(html))

print(f'Golden dataset — {len(golden_data)} recenzji:')
show_golden_dataset(golden_data)

## 🏗️ Model Pydantic (z CoT z Iteracji 3)

In [ ]:
class ReviewClassificationFewShot(BaseModel):
    """Klasyfikacja z CoT i few-shot examples."""

    reasoning: str = Field(
        description="Analiza recenzji krok po kroku przed klasyfikacją"
    )
    categories: list[str] = Field(
        description=f"Finalna lista kategorii. Dozwolone: {CATEGORIES}"
    )

print("✅ Model zdefiniowany")

## 📝 Definiowanie przykładów

> 🚧 **TODO:** Uzupełnij przykłady po analizie danych i przygotowaniu golden dataset.
>
> **Wskazówka:** Przykłady powinny:
> - Pokrywać różne typy recenzji
> - Zawierać trudne przypadki (wieloznaczne, graniczne)
> - Pokazywać recenzje z 1 kategorią I z wieloma kategoriami

In [ ]:
# ============================================================
# FEW-SHOT EXAMPLES
# TODO: Uzupełnij po analizie danych
# Format: lista słowników {"review": str, "categories": list[str]}
# ============================================================

FEW_SHOT_EXAMPLES = [
    # Przykładowy format (zastąp prawdziwymi przykładami):
    # {
    #     "review": "The game crashes every time I try to enter a new area...",
    #     "categories": ["bug"]
    # },
    # {
    #     "review": "FPS drops to 20 in crowded areas, unplayable on my RTX 3080",
    #     "categories": ["performance"]
    # },
    # {
    #     "review": "Beautiful world, but the story feels rushed and shallow characters",
    #     "categories": ["story", "graphics"]
    # },
]

print(f"Zdefiniowano {len(FEW_SHOT_EXAMPLES)} przykładów.")
if not FEW_SHOT_EXAMPLES:
    print("⚠️  Lista jest pusta - uzupełnij FEW_SHOT_EXAMPLES!")

In [ ]:
def build_few_shot_examples_text(examples: list[dict]) -> str:
    """Buduje fragment promptu z przykładami few-shot."""
    if not examples:
        return ""

    text = "\nPRZYKŁADY POPRAWNYCH KLASYFIKACJI:\n"
    for i, ex in enumerate(examples, 1):
        cats = ", ".join(ex["categories"])
        text += f"\n[Przykład {i}]\n"
        text += f"Recenzja: {ex['review']}\n"
        text += f"Kategorie: {cats}\n"
    return text


examples_text = build_few_shot_examples_text(FEW_SHOT_EXAMPLES)

SYSTEM_PROMPT_FEW_SHOT = f"""Jesteś ekspertem od analizy recenzji gier komputerowych.
Klasyfikujesz recenzje do kategorii tematycznych.

Dostępne kategorie: {categories_str}
{examples_text}
Analizuj każdą recenzję krok po kroku, a następnie podaj kategorie."""

USER_PROMPT = "Recenzja do sklasyfikowania: {review_text}"

print("System prompt:")
print(SYSTEM_PROMPT_FEW_SHOT)

## 🧪 Trzy warianty: Zero-shot vs One-shot vs Few-shot

Porównamy 3 próby jednocześnie - to jest właśnie ta funkcja `compare_trials()`!

In [ ]:
from tqdm.notebook import tqdm


def run_classification(system_prompt: str, texts: list[str]) -> list[list[str]]:
    """Uruchamia klasyfikację dla listy recenzji."""
    predictions = []
    for text in tqdm(texts):
        try:
            result = client.chat.completions.create(
                model=MODEL_NAME,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": USER_PROMPT.format(review_text=text)},
                ],
                response_model=ReviewClassificationFewShot,
                temperature=0.0,
            )
            predictions.append(result.categories)
        except Exception as e:
            print(f"Błąd: {e}")
            predictions.append([])
    return predictions


texts_to_classify = golden_texts if golden_texts else review_texts[:10]

# Wariant 1: Zero-shot (bez przykładów)
SYSTEM_ZERO_SHOT = f"""Jesteś ekspertem od analizy recenzji gier.
Klasyfikuj recenzje do kategorii. Dostępne: {categories_str}
Analizuj krok po kroku."""

# Wariant 2: One-shot (1 przykład)
one_shot_ex = FEW_SHOT_EXAMPLES[:1] if FEW_SHOT_EXAMPLES else []
SYSTEM_ONE_SHOT = SYSTEM_ZERO_SHOT + build_few_shot_examples_text(one_shot_ex)

# Wariant 3: Few-shot (wszystkie przykłady)
SYSTEM_FEW_SHOT = SYSTEM_PROMPT_FEW_SHOT

print("Uruchamiam 3 warianty klasyfikacji...")
print("\n[1/3] Zero-shot...")
preds_zero = run_classification(SYSTEM_ZERO_SHOT, texts_to_classify)

print("\n[2/3] One-shot...")
preds_one = run_classification(SYSTEM_ONE_SHOT, texts_to_classify)

print("\n[3/3] Few-shot...")
preds_few = run_classification(SYSTEM_FEW_SHOT, texts_to_classify)

print("\n✅ Wszystkie warianty gotowe!")

## 📊 Porównanie wszystkich wariantów

In [ ]:
if golden_labels:
    trial_zero = evaluate_trial(
        trial_name=f"Zero-shot",
        model=MODEL_NAME, prompt_variant="zero-shot + CoT",
        predictions=preds_zero, expected=golden_labels,
        review_texts=golden_texts, strategy=MatchStrategy.CONTAINS_ALL,
    )

    trial_one = evaluate_trial(
        trial_name=f"One-shot",
        model=MODEL_NAME, prompt_variant="one-shot + CoT",
        predictions=preds_one, expected=golden_labels,
        review_texts=golden_texts, strategy=MatchStrategy.CONTAINS_ALL,
    )

    trial_few = evaluate_trial(
        trial_name=f"Few-shot ({len(FEW_SHOT_EXAMPLES)} przykładów)",
        model=MODEL_NAME, prompt_variant="few-shot + CoT",
        predictions=preds_few, expected=golden_labels,
        review_texts=golden_texts, strategy=MatchStrategy.CONTAINS_ALL,
    )

    # Porównanie - tabela + wykres
    compare_trials([trial_zero, trial_one, trial_few])

## 🏆 Porównanie WSZYSTKICH iteracji

Zestawienie accuracy od Iteracji 1 do Iteracji 4.

In [ ]:
# ============================================================
# Załaduj wyniki z poprzednich iteracji
# Uzupełnij trial1, trial2, trial3 z poprzednich notebooków
# lub uruchom re-klasyfikację tutaj
# ============================================================

# Przykład jak połączyć wszystkie iteracje:
# all_trials = [trial1, trial2, trial3, trial_few]
# compare_trials(all_trials[:3])  # max 3 naraz w compare_trials

# Możesz też pokazać tabelę ręcznie:
import pandas as pd

# TODO: Uzupełnij po zakończeniu wszystkich iteracji
results_summary = pd.DataFrame([
    {"Iteracja": "1 - Zero-shot (raw)", "Podejście": "basic prompt", "Accuracy": "TODO%"},
    {"Iteracja": "2 - Structured Output", "Podejście": "Pydantic + Instructor", "Accuracy": "TODO%"},
    {"Iteracja": "3 - Chain-of-Thought", "Podejście": "CoT + Structured", "Accuracy": "TODO%"},
    {"Iteracja": "4 - Few-shot", "Podejście": f"Few-shot ({len(FEW_SHOT_EXAMPLES)} ex) + CoT", "Accuracy": "TODO%"},
])

print("PODSUMOWANIE WSZYSTKICH ITERACJI:")
print(results_summary.to_string(index=False))

## 💡 Ćwiczenie: Wybierz najlepszy prompt

### Zadanie:
Połącz wszystko czego się nauczyłeś - napisz prompt który osiąga najwyższą accuracy.

**Możesz:**
1. Zmienić model (`MODEL_NAME`)
2. Dodać więcej/lepszych przykładów
3. Ulepszył instrukcje CoT
4. Przetestować różne strategie matchowania (`EXACT`, `CONTAINS_ANY`, `CONTAINS_ALL`)

**🏆 Kto ma najwyższą accuracy wygrywa!**

In [ ]:
# ============================================================
# 🖊️  ĆWICZENIE: Twój najlepszy prompt
# ============================================================

MY_BEST_SYSTEM_PROMPT = """
# TODO: Napisz swój najlepszy prompt kombinując wszystkie techniki
"""

MY_FEW_SHOT_EXAMPLES = [
    # TODO: Dodaj najlepsze przykłady
]

# Uruchom i ewaluuj:
# my_preds = run_classification(MY_BEST_SYSTEM_PROMPT, texts_to_classify)
# my_trial = evaluate_trial(
#     trial_name="Mój najlepszy prompt",
#     model=MODEL_NAME,
#     prompt_variant="custom best",
#     predictions=my_preds,
#     expected=golden_labels,
#     review_texts=golden_texts,
#     strategy=MatchStrategy.CONTAINS_ALL,
# )
# my_trial.display()

print("Uzupełnij MY_BEST_SYSTEM_PROMPT i przetestuj!")

---

## 🏆 Podsumowanie całego warsztatu

### Czego się nauczyłeś:

| Iteracja | Technika | Kluczowy koncept |
|----------|----------|-----------------|
| 1 | Zero-shot prompting | System/user prompt, anatoma zapytania API |
| 2 | Structured Output | Pydantic BaseModel, Instructor, walidacja |
| 3 | Chain-of-Thought | reasoning → categories, "myślenie" modelu |
| 4 | Few-shot learning | Przykłady w prompcie, dobór przykładów |

### Kluczowe zasady:

1. **Zawsze waliduj output** - Pydantic + Instructor = brak niespodzianek
2. **Testuj iteracyjnie** - zacznij prosto (zero-shot), dodawaj złożoność tylko gdy potrzebna
3. **Mierz dokładnie** - golden dataset + ewaluacja = prawdziwy postęp, nie tylko "wydaje mi się że lepiej"
4. **Analizuj błędy** - patrz CO model klasyfikuje źle, nie tylko ILE
5. **Dokumentuj eksperymenty** - nadawaj nazwy próbom, zapisuj wyniki

### Dalsze kroki:
- **Fine-tuning** - gdy few-shot nie wystarcza i masz 100+ przykładów
- **RAG** - gdy potrzebujesz zewnętrznej wiedzy w odpowiedziach
- **Agents** - gdy zadanie wymaga wielokrokowego rozumowania i narzędzi